In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}",
)

prompt = template.format(country_a="Korea", country_b="USA")


                

In [6]:
chat = ChatOpenAI(model="gpt-4o-mini")
b = chat.predict("How many planets are there?", max_tokens=150)
b

'As of now, there are eight recognized planets in our solar system. They are:\n\n1. Mercury\n2. Venus\n3. Earth\n4. Mars\n5. Jupiter\n6. Saturn\n7. Uranus\n8. Neptune\n\nAdditionally, there are thousands of exoplanets (planets outside our solar system) that have been discovered in various star systems, with ongoing research and discoveries continuing to expand that number. The exact total of exoplanets is constantly changing as new ones are found and confirmed.'

In [7]:
chat.predict(prompt)

'The distance between Korea and the USA varies depending on the specific locations you are considering. For example:\n\n- The distance from Seoul, South Korea, to Los Angeles, California, is approximately 5,900 miles (about 9,500 kilometers) when flying directly.\n- The distance from Seoul to New York City is roughly 6,800 miles (about 10,900 kilometers) by air.\n\nThese distances can differ based on the specific cities chosen and the flight path taken.'

In [12]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(
        content="You are a geography expert. And you only reply in Korean.",
    ),
    AIMessage(content="안녕하세요. 내 이름은 JH 입니다."),
    HumanMessage(
        content="What is the distance between Korea and USA. Also, what is your name?",
    ),
]

chat.predict_messages(messages) 

AIMessage(content='한국과 미국 사이의 거리는 대략 10,000킬로미터 정도입니다. 하지만 정확한 거리는 출발지와 도착지에 따라 달라질 수 있습니다. 제 이름은 JH입니다.')

In [16]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "안녕하세요. 내 이름은{name} 입니다."),
        ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?"),
    ],
)

prompt = template.format_messages(
    language="Korean",
    name="JH",
    country_a="Korea",
    country_b="USA",
)

chat.predict_messages(prompt)

AIMessage(content='한국과 미국 간의 거리는 대략 10,000킬로미터 정도입니다. 제 이름은 JH입니다.')

In [18]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

p = CommaOutputParser()

p.parse("Hello, how, are, you")

['Hello', 'how', 'are', 'you']

In [24]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items}. Do NOT reply with anything else."),
    ("human", "{question}")
])

prompt = template.format_messages(
    max_items=10,
    question="What are the planets?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)


['Mercury',
 'Venus',
 'Earth',
 'Mars',
 'Jupiter',
 'Saturn',
 'Uranus',
 'Neptune',
 'Pluto',
 'Eris']

In [26]:
chain = template | chat | CommaOutputParser()
chain.invoke({
    "max_items": 5,
    "question": "What are the pokemons?"
})

['Pikachu', 'Charizard', 'Bulbasaur', 'Squirtle', 'Jigglypuff']